In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """简单的ConvNet

    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 输入大小（MNIST的情况下为784）
    hidden_size_list : 隐藏层的神经元数量的列表（e.g. [100, 100, 100]）
    output_size : 输出大小（MNIST的情况下为10）
    activation : 'relu' or 'sigmoid'
    weight_init_std : 指定权重的标准差（e.g. 0.01）
        指定'relu'或'he'的情况下设定“He的初始值”
        指定'sigmoid'或'xavier'的情况下设定“Xavier的初始值”
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 生成层
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """求损失函数
        参数x是输入数据、t是教师标签
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """求梯度（数值微分）

        Parameters
        ----------
        x : 输入数据
        t : 教师标签

        Returns
        -------
        具有各层的梯度的字典变量
            grads['W1']、grads['W2']、...是各层的权重
            grads['b1']、grads['b2']、...是各层的偏置
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """求梯度（误差反向传播法）

        Parameters
        ----------
        x : 输入数据
        t : 教师标签

        Returns
        -------
        具有各层的梯度的字典变量
            grads['W1']、grads['W2']、...是各层的权重
            grads['b1']、grads['b2']、...是各层的偏置
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [2]:
from common.optimizer import *

In [21]:
testcnn = SimpleConvNet();
losslist= []
optimizer = SGD(lr=0.001);

In [4]:
from dataset.mnist import load_mnist

In [5]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)
#导入训练数据和测试数据
x_train_1=x_train[:100]
t_train_1=t_train[:100]

In [ ]:
t_train_1[0]

In [ ]:
testcnn.gradient(x_train_1,t_train_1)

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist

from common.trainer import Trainer

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

In [ ]:


# 读入数据


# 处理花费时间较长的情况下减少数据 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 保存参数
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 绘制图形
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


In [ ]:
for i in range(1000):
    batch_mask = np.random.choice(60000,100);
    x_train_batch = x_train[batch_mask];
    t_train_batch = t_train[batch_mask];  #每一次学习都随机选300个数据
    
    grads = testcnn.gradient(x_train_batch,t_train_batch);
    losslist.append(testcnn.loss(x_train_batch,t_train_batch));
    optimizer.update(testcnn.params,grads)
    
    if i % 100 ==0:
        print(str(i) + '|' + str(losslist[-1]))

In [ ]:
batch_mask = np.random.choice(60000,100);
x_train_batch = x_train[batch_mask];
t_train_batch = t_train[batch_mask];  #每一次学习都随机选300个数据
grads = testcnn.gradient(x_train_batch,t_train_batch);
losslist.append(testcnn.loss(x_train_batch,t_train_batch));
optimizer.update(testcnn.params,grads)

In [28]:
for i in range(100):
    batch_mask = np.random.choice(60000,100);
    x_train_batch = x_train[batch_mask];
    t_train_batch = t_train[batch_mask];  #每一次学习都随机选300个数据
    
    grads = testcnn.gradient(x_train_batch,t_train_batch);
    losslist.append(testcnn.loss(x_train_batch,t_train_batch));
    optimizer.update(testcnn.params,grads)

In [25]:
bg = testcnn.params['W3'][0]
print(bg)

[-0.00091305  0.00337551 -0.01330463 -0.00678448  0.00736376 -0.00768928
  0.00087228 -0.02786747  0.0136771  -0.01731839]


In [30]:
fh = testcnn.params['W3'][0]
print(fh)

[-0.0009132   0.00331962 -0.01334951 -0.00676246  0.00738    -0.00765608
  0.0009086  -0.02786991  0.01365811 -0.01730382]


In [31]:
grads

{'W1': array([[[[-4.09488801e-05, -1.73724287e-05,  2.21652066e-04,
           -4.78420132e-05, -3.16892325e-04],
          [ 2.84609311e-04,  3.58848944e-04,  4.06987287e-04,
            1.83147369e-06, -1.22589396e-04],
          [ 5.85737539e-04,  3.16492535e-04,  1.30615173e-04,
           -4.29995781e-05, -4.33448003e-05],
          [ 5.76372017e-04,  4.02156565e-04,  1.50319902e-04,
            1.19869270e-04,  3.16273840e-04],
          [ 7.87909735e-04,  5.64798810e-04,  6.44808560e-04,
            6.14174692e-04,  8.82911509e-04]]],
 
 
        [[[ 6.49346788e-04,  9.55761718e-04,  8.69311061e-04,
            6.15892481e-04,  2.65963592e-04],
          [ 4.87196505e-04,  4.95195727e-04,  3.27918928e-04,
            1.11094619e-04,  2.34589961e-04],
          [ 6.38011595e-05, -1.39007795e-04, -1.97223809e-04,
           -2.68829795e-04, -2.45905240e-04],
          [ 6.81020868e-05,  2.54370049e-05, -2.19351232e-04,
           -4.80534607e-04, -4.82168992e-04],
          [ 7.81